In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from keras.utils import np_utils

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test_data = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
submission_format = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
a=train_data['Category'].value_counts()
a.shape

In [ ]:
submission_format.head()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data['Dates'].head()

In [ ]:
train_data['Dates'] = pd.to_datetime(train_data['Dates']).dt.time
test_data['Dates'] = pd.to_datetime(test_data['Dates']).dt.time

In [ ]:
train_data['Dates'] = train_data['Dates'].astype(str)
test_data['Dates'] = test_data['Dates'].astype(str)

In [ ]:
train_data['Dates'] = train_data['Dates'].replace(':','', regex=True)
test_data['Dates'] = test_data['Dates'].replace(':','', regex=True)

In [ ]:
train_data['Dates'] = train_data['Dates'].astype(int)
test_data['Dates'] = test_data['Dates'].astype(int)

In [ ]:
train_data.drop('Descript', axis = 1, inplace = True)

In [ ]:
# dow_dummies = pd.get_dummies(train_data.DayOfWeek)
# train_data = pd.concat([train_data, dow_dummies], axis=1)
# train_data.drop('DayOfWeek',axis=1,inplace=True)

In [ ]:
train_data['DayOfWeek'].value_counts()

In [ ]:
test_data['DayOfWeek'].value_counts()

In [ ]:
train_data['PdDistrict'].value_counts()

In [ ]:
test_data['PdDistrict'].value_counts()

In [ ]:
train_data.drop('Resolution', axis=1, inplace=True)

In [ ]:
not_categorical=train_data.drop(['Category'],axis=1).columns[train_data.drop(['Category'],axis=1).dtypes!='object']
categorical=train_data.drop(['Category'],axis=1).columns[train_data.drop(['Category'],axis=1).dtypes=='object']

In [ ]:
# for i in categorical:
    
#     dummies=pd.get_dummies(train_data[i])
#     train_data=pd.concat([train_data,dummies],axis=1)
#     train_data.drop(i,inplace=True,axis=1)
    
#     dummies=pd.get_dummies(test_data[i])
#     test_data=pd.concat([test_data,dummies],axis=1)
#     test_data.drop(i,inplace=True,axis=1)

In [ ]:
dow_dummies = pd.get_dummies(train_data.DayOfWeek)
train_data = pd.concat([train_data, dow_dummies], axis=1)
train_data.drop('DayOfWeek',axis=1,inplace=True)

In [ ]:
dow_dummies = pd.get_dummies(test_data.DayOfWeek)
test_data = pd.concat([test_data, dow_dummies], axis=1)
test_data.drop('DayOfWeek',axis=1,inplace=True)

In [ ]:
pd_dummies = pd.get_dummies(train_data.PdDistrict)
train_data = pd.concat([train_data, pd_dummies], axis=1)
train_data.drop('PdDistrict',axis=1,inplace=True)

In [ ]:
pd_dummies = pd.get_dummies(test_data.PdDistrict)
test_data = pd.concat([test_data, pd_dummies], axis=1)
test_data.drop('PdDistrict',axis=1,inplace=True)

In [ ]:
train_data.drop('Address', axis=1, inplace=True)
test_data.drop('Address', axis=1, inplace=True)

In [ ]:
test_data.shape

In [ ]:
train_data.shape

In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
X=train_data.drop(['Category'],axis=1)
y=pd.get_dummies(train_data['Category'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)

In [ ]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from tensorflow import keras

from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential

In [ ]:
model=Sequential()

model.add(Dense(256,input_shape=(X.shape[1],)))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(39))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.fit(X_train,y_train,
         batch_size=256,
         epochs=10,
         verbose=2,
         validation_data=(X_test,y_test))

In [ ]:
pred=model.predict(test_data.drop('Id',axis=1))

In [ ]:
pred_final=[np.argmax(i) for i in pred]

In [ ]:
predreshape = np.max(pred, axis=1).reshape(-1, 1)
pred_final = np.array((pred == predreshape), dtype='int32')
pred_final

In [ ]:
column=list(submission_format.columns)
column.remove('Id')
print(column)

In [ ]:
Submission=pd.DataFrame({'Id':test_data['Id']})
for i,v in enumerate(column):
    Submission[v]=pred_final[:,i]
    
Submission.head()

In [ ]:
Submission.to_csv('Submission.csv', index=False)